In [1]:
class Board():

    # list of all 8 directions on the board, as (x,y) offsets
    __directions = [(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1),(0,1)]

    def __init__(self):
        "Set up initial board configuration."

        self.n = 4

        # Create the empty board array.
        self.pieces = [[0 for y in range(self.n)] for x in range(self.n)]

        # Set up the initial Amazons
        self.pieces[1][0] = 1
        self.pieces[0][2] = 1
        self.pieces[2][3] = -1
        self.pieces[3][1] = -1
            
    # add [][] indexer syntax to the Board
    def __getitem__(self, index): 
        return self.pieces[index]
    
    def count_diff(self, color):
        count = 0
        for x in range(self.n):
            for y in range(self.n):
                if self[x][y] == color:
                    count += 1
                if self[x][y] == -color:
                    count -= 1
        return count
    
    def pack_move(self, x1, y1, x2, y2, x3, y3):
        return '-'.join([chr(x1+65)+str(y1+1), chr(x2+65)+str(y2+1), chr(x3+65)+str(y3+1)])

    def unpack_move(self, move):
        coords = move.split('-')
        d = []
        for c in coords:
            d.append(ord(c[0])-65)
            d.append(int(c[1:])-1)
        return d
    
    def expand_moves(self, x1, y1):
        valid_coords = set(range(self.n))
        
        assert(x1 in valid_coords)
        assert(y1 in valid_coords)
        assert(self[x1][y1] in (-1, 1))

        moves = []
        
        for x2_delta, y2_delta in self.__directions:
            x2 = x1
            y2 = y1
            while True:
                x2 += x2_delta
                y2 += y2_delta
                
                if x2 not in valid_coords or y2 not in valid_coords:
                    break
                if self[x2][y2] != 0:
                    break
                    
                for x3_delta, y3_delta in self.__directions:
                    x3 = x2
                    y3 = y2
                    while True:
                        x3 += x3_delta
                        y3 += y3_delta
                        
                        if x3 not in valid_coords or y3 not in valid_coords:
                            break
                        if self[x3][y3] != 0 and not (x3 == x1 and y3 == y1):
                            break
                            
                        moves.append(self.pack_move(x1, y1, x2, y2, x3, y3))
        return moves
    
    def get_legal_moves(self, colour):
        moves = []
        for x in range(self.n):
            for y in range(self.n):
                if self[x][y] == colour:
                    moves += self.expand_moves(x, y)
        return moves
    
    def has_legal_moves(self, colour):
        for x in range(self.n):
            for y in range(self.n):
                if self[x][y] == colour:
                    if len(self.expand_moves(x, y)) > 0:
                        return True
        return False
    
    def execute_move(self, move, colour):
        x1, y1, x2, y2, x3, y3 = self.unpack_move(move)
        assert(self[x1][y1] == colour)
        self[x2][y2] = self[x1][y1]
        self[x1][y1] = 0
        self[x3][y3] = 2
        
    def display(self):
        pieces = np.copy(self.pieces)
        
        print("  -" + "-" * (3 * n) + "-")

        for y in range(n - 1, -1, -1):
            print("{} |".format(y + 1), end="")
            for x in range(self.n):
                piece = pieces[x][y]
                if piece == -1: print(" B ", end="")
                elif piece == 1: print(" W ", end="")
                elif piece == 2: print(" X ", end="")
                else: print(" . ", end="")
            print("|")

        print("  -" + "-" * (3 * n) + "-")
        print("   ", end="")
        for i in range(self.n):
            print(" {} ".format(chr(i + 65)), end="")

In [2]:
b = Board()

In [3]:
def is_valid_move(x1, y1, x2, y2):
    if x1 == x2 and y1 == y2:
        return False
    if x1 == x2 and y1 != y2:
        return True
    if x1 != x2 and y1 == y2:
        return True
    if (x2 - x1) == (y2 - y1):
        return True
    if (x2 - x1) == -1 * (y2 - y1):
        return True
    return False

def all_valid_moves(n=4):
    valid_moves = []
    for x1 in range(n):
        for y1 in range(n):
            for x2 in range(n):
                for y2 in range(n):
                    for x3 in range(n):
                        for y3 in range(n):
                            if is_valid_move(x1, y1, x2, y2) and is_valid_move(x2, y2, x3, y3):
                                valid_moves.append(b.pack_move(x1, y1, x2, y2, x3, y3))
    return valid_moves

In [4]:
packed = b.pack_move(0, 7, 1, 0, 11, 0)
x1, y1, x2, y2, x3, y3 = b.unpack_move(packed)
print(packed, x1, y1, x2, y2, x3, y3)

A8-B1-L1 0 7 1 0 11 0


In [5]:
valid_moves = all_valid_moves(4)

moves = []

colour = 1

for x in range(4):
    for y in range(4):
        if b[x][y] == colour:
            moves += b.expand_moves(x, y)
    
for move in moves:
    assert(move in valid_moves)
    print(move)

A3-B4-C3
A3-B4-B3
A3-B4-B2
A3-B4-A3
A3-B4-A4
A3-B3-C3
A3-B3-D3
A3-B3-C2
A3-B3-D1
A3-B3-B2
A3-B3-A2
A3-B3-A3
A3-B3-A4
A3-B3-B4
A3-C3-D4
A3-C3-D3
A3-C3-C2
A3-C3-C1
A3-C3-B2
A3-C3-A1
A3-C3-B3
A3-C3-A3
A3-C3-B4
A3-D3-C2
A3-D3-C3
A3-D3-B3
A3-D3-A3
A3-D3-D4
A3-B2-C3
A3-B2-D4
A3-B2-C2
A3-B2-C1
A3-B2-A1
A3-B2-A2
A3-B2-A3
A3-B2-B3
A3-B2-B4
A3-C1-D1
A3-C1-B2
A3-C1-A3
A3-C1-C2
A3-C1-C3
A3-A2-B3
A3-A2-B2
A3-A2-C2
A3-A2-A1
A3-A2-A3
A3-A2-A4
A3-A1-B2
A3-A1-C3
A3-A1-D4
A3-A1-A2
A3-A1-A3
A3-A1-A4
A3-A4-B4
A3-A4-B3
A3-A4-C2
A3-A4-D1
A3-A4-A3
A3-A4-A2
A3-A4-A1
B1-C2-D3
B1-C2-D1
B1-C2-C1
B1-C2-B1
B1-C2-B2
B1-C2-A2
B1-C2-B3
B1-C2-A4
B1-C2-C3
B1-D3-C2
B1-D3-B1
B1-D3-C3
B1-D3-B3
B1-D3-D4
B1-C1-D1
B1-C1-B1
B1-C1-A1
B1-C1-B2
B1-C1-C2
B1-C1-C3
B1-D1-C1
B1-D1-B1
B1-D1-A1
B1-D1-C2
B1-D1-B3
B1-D1-A4
B1-A1-B2
B1-A1-C3
B1-A1-D4
B1-A1-B1
B1-A1-C1
B1-A1-D1
B1-A1-A2
B1-A2-B3
B1-A2-B2
B1-A2-C2
B1-A2-B1
B1-A2-A1
B1-B2-C3
B1-B2-D4
B1-B2-C2
B1-B2-C1
B1-B2-B1
B1-B2-A1
B1-B2-A2
B1-B2-B3
B1-B2-B4
B1-B3-C3
B1-B3-D3
B1-B3-C2
B

In [23]:
move_dict = {}

for i in range(len(valid_moves)):
    move_dict[valid_moves[i]] = i
    move_dict[i] = valid_moves[i]
    
len(move_dict), len(valid_moves)

[i for i in move_dict.keys() if type(i) == str]

['A1-A2-A1',
 'A1-A2-A3',
 'A1-A2-A4',
 'A1-A2-B1',
 'A1-A2-B2',
 'A1-A2-B3',
 'A1-A2-C2',
 'A1-A2-C4',
 'A1-A2-D2',
 'A1-A3-A1',
 'A1-A3-A2',
 'A1-A3-A4',
 'A1-A3-B2',
 'A1-A3-B3',
 'A1-A3-B4',
 'A1-A3-C1',
 'A1-A3-C3',
 'A1-A3-D3',
 'A1-A4-A1',
 'A1-A4-A2',
 'A1-A4-A3',
 'A1-A4-B3',
 'A1-A4-B4',
 'A1-A4-C2',
 'A1-A4-C4',
 'A1-A4-D1',
 'A1-A4-D4',
 'A1-B1-A1',
 'A1-B1-A2',
 'A1-B1-B2',
 'A1-B1-B3',
 'A1-B1-B4',
 'A1-B1-C1',
 'A1-B1-C2',
 'A1-B1-D1',
 'A1-B1-D3',
 'A1-B2-A1',
 'A1-B2-A2',
 'A1-B2-A3',
 'A1-B2-B1',
 'A1-B2-B3',
 'A1-B2-B4',
 'A1-B2-C1',
 'A1-B2-C2',
 'A1-B2-C3',
 'A1-B2-D2',
 'A1-B2-D4',
 'A1-C1-A1',
 'A1-C1-A3',
 'A1-C1-B1',
 'A1-C1-B2',
 'A1-C1-C2',
 'A1-C1-C3',
 'A1-C1-C4',
 'A1-C1-D1',
 'A1-C1-D2',
 'A1-C3-A1',
 'A1-C3-A3',
 'A1-C3-B2',
 'A1-C3-B3',
 'A1-C3-B4',
 'A1-C3-C1',
 'A1-C3-C2',
 'A1-C3-C4',
 'A1-C3-D2',
 'A1-C3-D3',
 'A1-C3-D4',
 'A1-D1-A1',
 'A1-D1-A4',
 'A1-D1-B1',
 'A1-D1-B3',
 'A1-D1-C1',
 'A1-D1-C2',
 'A1-D1-D2',
 'A1-D1-D3',
 'A1-D1-D4',
 'A1-D4-A1',

In [16]:
b.get_legal_moves(1)

['A3-B3-C3',
 'A3-B3-D3',
 'A3-B3-C2',
 'A3-B3-D1',
 'A3-B3-B2',
 'A3-B3-A2',
 'A3-B3-A3',
 'A3-B3-A4',
 'A3-C3-D4',
 'A3-C3-D3',
 'A3-C3-C2',
 'A3-C3-C1',
 'A3-C3-B2',
 'A3-C3-A1',
 'A3-C3-B3',
 'A3-C3-A3',
 'A3-D3-C2',
 'A3-D3-C3',
 'A3-D3-B3',
 'A3-D3-A3',
 'A3-D3-D4',
 'A3-B2-C3',
 'A3-B2-D4',
 'A3-B2-C2',
 'A3-B2-C1',
 'A3-B2-A1',
 'A3-B2-A2',
 'A3-B2-A3',
 'A3-B2-B3',
 'A3-C1-D1',
 'A3-C1-B2',
 'A3-C1-A3',
 'A3-C1-C2',
 'A3-C1-C3',
 'A3-A2-B3',
 'A3-A2-B2',
 'A3-A2-C2',
 'A3-A2-A1',
 'A3-A2-A3',
 'A3-A2-A4',
 'A3-A1-B2',
 'A3-A1-C3',
 'A3-A1-D4',
 'A3-A1-A2',
 'A3-A1-A3',
 'A3-A1-A4',
 'A3-A4-B3',
 'A3-A4-C2',
 'A3-A4-D1',
 'A3-A4-A3',
 'A3-A4-A2',
 'A3-A4-A1',
 'B4-C3-D4',
 'B4-C3-D3',
 'B4-C3-C2',
 'B4-C3-C1',
 'B4-C3-B2',
 'B4-C3-A1',
 'B4-C3-B3',
 'B4-C3-B4',
 'B4-B3-C3',
 'B4-B3-D3',
 'B4-B3-C2',
 'B4-B3-D1',
 'B4-B3-B2',
 'B4-B3-A2',
 'B4-B3-A4',
 'B4-B3-B4',
 'B4-B2-C3',
 'B4-B2-D4',
 'B4-B2-C2',
 'B4-B2-C1',
 'B4-B2-A1',
 'B4-B2-A2',
 'B4-B2-B3',
 'B4-B2-B4',
 'B4-A4-B4',

In [7]:
import numpy as np
from random import choice
# print(np.rot90(np.array(b.pieces)))
# move = choice(moves)
# print('\n', move, '\n')
# b.executeMove(move)
# print(np.rot90(np.array(b.pieces)))

In [8]:
board = np.copy(b.pieces)

n = board.shape[0]

for y in range(n):
    print (y,"|",end="")
print("")
print(" -----------------------")
for y in range(n):
    print(y, "|",end="")    # print the row #
    for x in range(n):
        piece = board[y][x]    # get the piece to print
        if piece == -1: print("B ",end="")
        elif piece == 1: print("W ",end="")
        elif piece == 2: print("X ",end="")
        else:
            if x==n:
                print("-",end="")
            else:
                print("- ",end="")
    print("|")

print("   -----------------------")


0 |1 |2 |3 |
 -----------------------
0 |- - W - |
1 |W - - - |
2 |- - - B |
3 |- B - - |
   -----------------------


In [9]:
b.display()
b.execute_move("B1-B4-B1", 1)
print('\n')
b.display()

  --------------
4 | .  .  B  . |
3 | W  .  .  . |
2 | .  .  .  B |
1 | .  W  .  . |
  --------------
    A  B  C  D 

  --------------
4 | .  W  B  . |
3 | W  .  .  . |
2 | .  .  .  B |
1 | .  X  .  . |
  --------------
    A  B  C  D 

In [34]:
example_size = 4
example_board = np.array([
    [choice([-1, 0, 1, 2]) for i in range(example_size)] for j in range(example_size)
])

def decompose(board):
    white = np.zeros(board.shape)
    white[board == 1] = 1
    black = np.zeros(board.shape)
    black[board == -1] = 1
    arrows = np.zeros(board.shape)
    arrows[board == 2] = 1
    return np.stack((white, black, arrows), axis=0).astype(np.int64)

def recompose(board):
    recomposed = np.zeros(board[0].shape).astype(np.int64)
    recomposed[board[0] == 1] = 1
    recomposed[board[1] == 1] = -1
    recomposed[board[2] == 1] = 2
    return recomposed


print(example_board)
print()
print(decompose(example_board))
print()
print(recompose(decompose(example_board)))

[[ 2 -1  2  0]
 [ 2  1  0  2]
 [-1  1  1  1]
 [ 1  2 -1 -1]]

[[[0 0 0 0]
  [0 1 0 0]
  [0 1 1 1]
  [1 0 0 0]]

 [[0 1 0 0]
  [0 0 0 0]
  [1 0 0 0]
  [0 0 1 1]]

 [[1 0 1 0]
  [1 0 0 1]
  [0 0 0 0]
  [0 1 0 0]]]

[[ 2 -1  2  0]
 [ 2  1  0  2]
 [-1  1  1  1]
 [ 1  2 -1 -1]]
